한국어 오픈소스 LLM 테스트

In [7]:
'''
한국어 Fine Tune Llama3 : https://devmeta.tistory.com/80#google_vignette

hugging face safetensor -> gguf -> gguf 4bit : https://www.youtube.com/watch?v=jOEu0PE4ozM
'''

'\n한국어 Fine Tune Llama3 : https://devmeta.tistory.com/80#google_vignette\n\nhugging face safetensor -> gguf -> gguf 4bit : https://www.youtube.com/watch?v=jOEu0PE4ozM\n'

In [6]:
# 1. 모델 다운로드
from huggingface_hub import snapshot_download
snapshot_download(repo_id='MLP-KTLim/llama-3-Korean-Bllossom-8B', local_dir_use_symlinks=False)

Fetching 12 files: 100%|██████████| 12/12 [00:00<00:00, 48.19it/s]


'C:\\Users\\112fk\\.cache\\huggingface\\hub\\models--MLP-KTLim--llama-3-Korean-Bllossom-8B\\snapshots\\8a738f9f622ffc2b0a4a6b81dabbca80406248bf'

In [ ]:
# 2. GGUF F16으로 convert 시키기
'''
llama-ccp 레포지토리 : https://github.com/ggerganov/llama.cpp/

아나콘다 환경에서 문제 생겨서 wsl2에다가
- ubuntu22.04 설치
- venv 설치
- pytorch 설치 : pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
- torch.cuda.is_available() True 나오는 것 확인
- pip instal transformers 설치
- 위 코드 터미널 python에서 실행하여 모델들 다운로드(safetensors)
- 현재 모델 다운로드 받은 경로 복사 : /home/ubuntu/.cache/huggingface/hub/models--MLP-KTLim--llama-3-Korean-Bllossom-8B/snapshots/8a738f9f622ffc2b0a4a6b81dabbca80406248bf
- 들어가서 가상환경 새로 만들고(venv2) requirments.txt 설치
- python convert_hf_to_gguf.py 입력하여 명령어 조회
- 다음와 같이 수정된 명령어 입력하여 f16으로 변환 : python convert_hf_to_gguf.py --outtype f16 --verbose /path/to/your/model
- 해당 경로에 저장되었다고 나옴 : INFO:hf-to-gguf:Model successfully exported to /home/ubuntu/.cache/huggingface/hub/models--MLP-KTLim--llama-3-Korean-Bllossom-8B/snapshots/8a738f9f622ffc2b0a4a6b81dabbca80406248bf/ggml-model-f16.gguf

- 이제 4bit로 변환할 순서
- https://github.com/ggerganov/llama.cpp/releases 여기 들어가서 llama-b3369-bin-ubuntu-x64.zip 다운로드 받고 wsl2로 unzip해라. 그리고 해당 폴더 안으로 들어가서 아래 실행.

ubuntu@nyan-pc:~/llama-quantize$ ./build/bin/llama-quantize /home/ubuntu/.cache/huggingface/hub/models--MLP-KTLim--llama-3-Korean-Bllossom-8B/snaps
hots/8a738f9f622ffc2b0a4a6b81dabbca80406248bf/ggml-model-f16.gguf llama3-8b-4bit-q4_1.gguf Q4_1

- 이제 4bit 양자화가 제대로 되었다고 뜸.
- wsl2에 가상환경 하나 만들어서 ollama 설치하고 아래 블로그 링크 따라하면 잘 동작 한다. 하지만 템플릿이 잘못되었는지 이상함. 확실히 실행 속도는 빠르다.
- 하나의 디렉토리에 4bit 양자화된 gguf 파일이랑 토크나이저 넣고 Modelfile 다음 링크와 같이 만들어서 시도 : https://wooiljeong.github.io/ml/gguf-llm/

'''

Ollama랑 langchain 합쳐서 테스트 중

In [9]:
## 1. 기본 채팅
from langchain_community.llms import Ollama
llm = Ollama(model="llama3-ko")

In [10]:
input_txt = '긴 소설 하나를 써줄래?'
for chunk in llm.stream(input_txt):
    print(chunk, end='')

물론이죠! 어떤 주제로 이야기를 시작하고 싶으신가요? 판타지, 과학소설, 로맨스, 스릴러 등 다양한 장르 중에서 선택해주세요

In [11]:
from langchain_core.messages import HumanMessage
from langchain_core.messages import AIMessage

llm.invoke([HumanMessage(content="안녕. 나는 이언이야")])

'Assistant: 안녕하세요! 저는 당신의 AI 어시스턴트입니다. 무엇을 도와드릴까요'

In [12]:
llm.invoke(
    [
        HumanMessage(content="안녕. 나는 이언이야"),
        AIMessage(content="Assistant: 안녕하세요! 저는 당신의 AI 어시스턴트입니다. 무엇을 도와드릴까요"),
        HumanMessage(content="나 배고파 너무. 오늘 저녁에 무엇을 먹으면 좋을까?"),
    ]
)

'Assistant: 안녕하세요! 저녁 식사에 대해 조언해 드리겠습니다. 몇 가지 추천드립니다.\r\n1. **한식**: 비빔밥, 불고기, 김치찌개 등 다양한 한식을 선택할 수 있습니다.\n2. **양식**: 짜장면, 탕수육, 두부 등 맛있는 양식을 즐길 수 있습니다.\n3. **일식**: 스시, 라멘, 돈까스 등 일본 요리를 추천합니다.\n4. **중식**: 짬뽕이, 탕원, 마파두프 등 다양한 중식을 선택할 수 있습니다.\n\n어떤 종류의 음식을 선호하시는지 알려주시면 더 구체적인 추천을 드릴 수 있습니다'

In [14]:
llm.invoke([HumanMessage(content="내가 처음에 물어본 것 말이야")])

'Assistant: Ah, I apologize for not recalling your previous question. Could you please remind me what it was?'

메시지 히스토리 전달

In [21]:
from langchain_core.chat_history import (
    BaseChatMessageHistory,
    InMemoryChatMessageHistory,
)
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(llm, get_session_history)

In [22]:
config = {"configurable": {"session_id": "abc2"}}

In [23]:
response = with_message_history.invoke(
    [HumanMessage(content="안녕! 내 이름은 라이언이야")],
    config=config,
)

response

Error in RootListenersTracer.on_llm_end callback: KeyError('message')


'Assistant: 안녕하세요, 라이언! 어떻게 오늘 하루가 지내고 있나요'

In [20]:
response = with_message_history.invoke(
    [HumanMessage(content="내 이름이 뭐라고?")],
    config=config,
)

response

Error in RootListenersTracer.on_llm_end callback: KeyError('message')


'Assistant: 안녕하세요! 저는 [Your Name]입니다. 무엇을 도와드릴까요'

에러 나서 다른 방법으로 시도 : https://github.com/langchain-ai/langchain/issues/22060

In [26]:
from langchain_community.chat_models import ChatOllama
llm = ChatOllama(model="llama3-ko")

input_txt = '긴 소설 하나를 써줄래?'
for chunk in llm.stream(input_txt):
    print(chunk.content, end='')

물론이죠! 어떤 주제로 소설을 쓰고 싶으신가요? 판타지, 로맨스, 미스터리, SF 등 다양한 장르 중에서 선택해 주세요

In [28]:
from langchain_core.chat_history import (
    BaseChatMessageHistory,
    InMemoryChatMessageHistory,
)
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(llm, get_session_history)

In [30]:
config = {"configurable": {"session_id": "abc2"}}

response = with_message_history.invoke(
    [HumanMessage(content="안녕! 내 이름은 라이언이야")],
    config=config,
)

response.content

'안녕하세요, 라이언! 어떻게 도와드릴까요'

In [38]:
response = with_message_history.invoke(
    [HumanMessage(content="앞으로 내 이름은 김철수야")],
    config=config,
)

response.content

'알겠습니다, 김철수님! 앞으로도 도움이 필요하시면 언제든지 말씀해 주세요. 어떻게 도와드릴까요?\r\n\r\n(참고로, 라이언이라는 이름을 기억하고 계시군요)'

RAG 추가 구현해보기